In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import pickle
import numpy as np
import time
import random
from collections import defaultdict

import torch.nn.functional as F
import torch.utils.data
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt
import datetime
import argparse
import os

In [2]:
    parser = argparse.ArgumentParser(description='Social Recommendation: GraphRec model')
    parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
    parser.add_argument('--embed_dim', type=int, default=64, metavar='N', help='embedding size')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR', help='learning rate')
    parser.add_argument('--test_batch_size', type=int, default=1000, metavar='N', help='input batch size for testing')
    parser.add_argument('--epochs', type=int, default=100, metavar='N', help='number of epochs to train')
    args = parser.parse_args(args=[])

In [3]:
device = torch.device( "cpu")

In [4]:
embed_dim = args.embed_dim
dir_data = './data/toy_dataset'
path_data = dir_data + ".pickle"
data_file = open(path_data, 'rb')

history_u_lists, 

history_ur_lists, 

history_v_lists, 

history_vr_lists, 

train_u,  14091

train_v, 

train_r, 

test_u, 

test_v, 

test_r, 

social_adj_lists, set集合，{0:{1,2,5,8},1:{0,5,8}}

ratings_list 

In [5]:
history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(data_file)

In [6]:
torch.LongTensor(train_u)
# len(train_u)

tensor([681,  81, 172,  ..., 518, 377, 479])

In [7]:
import pandas as pd


# history_u_lists, history_ur_lists:
# 用户的购买历史记录（在培训集中设置的项目）以及他/她的评分得分（dict）
# history_v_lists, history_vr_lists
# 与项目互动的用户集（在培训集中）和评分分数（dict）

# train_u, train_v, train_r: 训练集 (user, item, rating)
# test_u, test_v, test_r: 测试集 (user, item, rating)

# social_adj_lists: user相邻的邻居
# ratings_list: 评分值从0.5到4.0（8个维度嵌入）

In [8]:
# 三个元素的元祖的集合  是用户u对项目v的打分r   14091*()
trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                              torch.FloatTensor(train_r))

train_loader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True)

In [9]:
len(trainset)

14091

In [10]:
for i, data in enumerate(train_loader):
    print(trainset.__getitem__(i))
i

(tensor(681), tensor(0), tensor(2.))
(tensor(81), tensor(1), tensor(1.))
(tensor(172), tensor(2), tensor(3.))
(tensor(83), tensor(3), tensor(3.))
(tensor(151), tensor(4), tensor(3.))
(tensor(27), tensor(5), tensor(4.))
(tensor(197), tensor(6), tensor(2.5000))
(tensor(316), tensor(7), tensor(3.5000))
(tensor(385), tensor(8), tensor(2.))
(tensor(231), tensor(9), tensor(3.5000))
(tensor(646), tensor(10), tensor(2.))
(tensor(278), tensor(11), tensor(4.))
(tensor(23), tensor(12), tensor(3.))
(tensor(477), tensor(13), tensor(3.5000))
(tensor(392), tensor(14), tensor(3.5000))
(tensor(145), tensor(15), tensor(3.5000))
(tensor(35), tensor(16), tensor(2.5000))
(tensor(292), tensor(17), tensor(4.))
(tensor(74), tensor(18), tensor(1.5000))
(tensor(669), tensor(19), tensor(4.))
(tensor(528), tensor(20), tensor(3.5000))
(tensor(522), tensor(21), tensor(3.))
(tensor(590), tensor(22), tensor(1.5000))
(tensor(552), tensor(23), tensor(1.5000))
(tensor(336), tensor(20), tensor(3.5000))
(tensor(676), tens

110

In [11]:
for i, data in enumerate(train_loader):
    print(train_loader.dataset.__getitem__(i))
i

(tensor(681), tensor(0), tensor(2.))
(tensor(81), tensor(1), tensor(1.))
(tensor(172), tensor(2), tensor(3.))
(tensor(83), tensor(3), tensor(3.))
(tensor(151), tensor(4), tensor(3.))
(tensor(27), tensor(5), tensor(4.))
(tensor(197), tensor(6), tensor(2.5000))
(tensor(316), tensor(7), tensor(3.5000))
(tensor(385), tensor(8), tensor(2.))
(tensor(231), tensor(9), tensor(3.5000))
(tensor(646), tensor(10), tensor(2.))
(tensor(278), tensor(11), tensor(4.))
(tensor(23), tensor(12), tensor(3.))
(tensor(477), tensor(13), tensor(3.5000))
(tensor(392), tensor(14), tensor(3.5000))
(tensor(145), tensor(15), tensor(3.5000))
(tensor(35), tensor(16), tensor(2.5000))
(tensor(292), tensor(17), tensor(4.))
(tensor(74), tensor(18), tensor(1.5000))
(tensor(669), tensor(19), tensor(4.))
(tensor(528), tensor(20), tensor(3.5000))
(tensor(522), tensor(21), tensor(3.))
(tensor(590), tensor(22), tensor(1.5000))
(tensor(552), tensor(23), tensor(1.5000))
(tensor(336), tensor(20), tensor(3.5000))
(tensor(676), tens

110

In [12]:
testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                             torch.FloatTensor(test_r))

test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=True)

In [13]:
num_users = history_u_lists.__len__() #705 users
num_items = history_v_lists.__len__()  #1941 item
num_ratings = ratings_list.__len__()  #8  8种分数

In [14]:
u2e = nn.Embedding(num_users, embed_dim).to(device)
v2e = nn.Embedding(num_items, embed_dim).to(device)

# 将评分也转化成Embedding
r2e = nn.Embedding(num_ratings, embed_dim).to(device)

In [15]:
v2e.weight

Parameter containing:
tensor([[ 0.0408,  1.2600, -1.9267,  ..., -1.2854, -1.6544, -0.3633],
        [-2.5353, -2.4534, -0.2431,  ...,  0.5663, -0.1036, -1.1639],
        [-0.6861, -0.5613,  0.6603,  ...,  1.0003, -0.4729, -0.0249],
        ...,
        [-0.4095, -1.1725, -0.0697,  ...,  0.4732,  0.9023,  1.8954],
        [ 0.9798,  0.2345, -0.0410,  ...,  1.0442,  1.3068, -0.4143],
        [-0.4071, -0.0628, -0.7700,  ..., -0.2350,  1.0591,  0.3958]],
       requires_grad=True)

In [16]:
num_ratings

8

### Attention

In [17]:
class Attention(nn.Module):
    def __init__(self, embedding_dims):
        super(Attention, self).__init__()
        self.embed_dim = embedding_dims
#         进行线性变换   (N*64) *A * (N * 64) + b
        self.bilinear = nn.Bilinear(self.embed_dim, self.embed_dim, 1)
        self.att1 = nn.Linear(self.embed_dim * 2, self.embed_dim)   #128*64
        self.att2 = nn.Linear(self.embed_dim, self.embed_dim)   # 64*64
        self.att3 = nn.Linear(self.embed_dim, 1)   # 64*1
        self.softmax = nn.Softmax(0)

    def forward(self, node1, u_rep, num_neighs):
        uv_reps = u_rep.repeat(num_neighs, 1)
        x = torch.cat((node1, uv_reps), 1)
        x = F.relu(self.att1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.att2(x))
        x = F.dropout(x, training=self.training)
        x = self.att3(x)
        att = F.softmax(x, dim=0)
        return att

In [18]:
r2e.weight[1]

tensor([ 0.5792, -0.6137,  1.0095, -0.6832,  0.2060, -1.2658,  1.1402,  0.4925,
        -0.0717, -0.6354, -0.1669, -0.3147,  0.0313,  2.0650, -0.6001, -0.5757,
         0.6481,  0.5077, -0.8498,  0.0419, -0.3678, -0.1515,  1.4606,  0.0768,
        -0.0970, -1.1362,  0.4450, -0.0806, -0.5280, -1.7293, -0.1698,  0.0600,
         0.3642,  0.1660,  1.0511, -0.2953,  0.5412,  1.5216,  0.9889,  0.2657,
        -1.2285,  1.3287,  0.0948,  0.4276,  0.2429, -0.7598,  0.3844, -0.6880,
        -0.4076, -0.0760,  0.1681,  1.5098,  0.0557,  1.8551, -0.4345,  0.5974,
        -0.6801, -1.5289, -1.3795, -0.2088, -1.0530,  1.6212,  0.0908,  1.0095],
       grad_fn=<SelectBackward>)

### UV_Aggregator

UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)

In [19]:

class UV_Aggregator(nn.Module):
    """
    item and user aggregator: for aggregating embeddings of neighbors (item/user aggreagator).
    """

    def __init__(self, v2e, r2e, u2e, embed_dim, cuda="cpu", uv=True):
        super(UV_Aggregator, self).__init__()
        self.uv = uv
        self.v2e = v2e
        self.r2e = r2e
        self.u2e = u2e
        self.device = cuda
        self.embed_dim = embed_dim  #64
#         128 * 64
        self.w_r1 = nn.Linear(self.embed_dim * 2, self.embed_dim)  
#         64*64
        self.w_r2 = nn.Linear(self.embed_dim, self.embed_dim)
#         64*1
        self.att = Attention(self.embed_dim)
        
    def forward(self, nodes, history_uv, history_r):
#         随机生成的embedding 矩阵    送到内存中
        embed_matrix = torch.empty(len(history_uv), self.embed_dim, dtype=torch.float).to(self.device)
        
        for i in range(len(history_uv)):
#             对于每一条记录
            history = history_uv[i]
#             每条记录的长度
            num_histroy_item = len(history)
#             标签
            tmp_label = history_r[i]
            
            if self.uv == True:
                # user component
#                 去history个items的嵌入向量
                e_uv = self.v2e.weight[history]
#                   取history个user的嵌入向量
                uv_rep = self.u2e.weight[nodes[i]]
            else:
                # item component
                e_uv = self.u2e.weight[history]
                uv_rep = self.v2e.weight[nodes[i]]
#             将具体打分转化成向量
            e_r = self.r2e.weight[tmp_label]
#             user 和 item 向量拼接
            x = torch.cat((e_uv, e_r), 1)
            x = F.relu(self.w_r1(x))
            o_history = F.relu(self.w_r2(x))
#             求软注意力机制的 alpha
            att_w = self.att(o_history, uv_rep, num_histroy_item)
#             执行 mat1 和 mat2 的矩阵乘法.
#             gv() 为两层神经网络
            att_history = torch.mm(o_history.t(), att_w)
            att_history = att_history.t()
            
#             att_history:  1*64
            embed_matrix[i] = att_history
        to_feats = embed_matrix
        return to_feats

In [20]:
torch.empty(5, 3)

tensor([[-2.8494e-36, -9.0072e+15,  7.1555e+22],
        [ 5.5158e-14,  9.3853e+09,  8.8544e+14],
        [ 4.9485e+04,  2.2034e-10,  2.5353e+30],
        [ 6.4460e-44,  3.2647e+03,  2.1737e+05],
        [ 8.4431e+08,  6.5613e-39,  8.3971e-32]])

### UV_Encoder

In [21]:
class UV_Encoder(nn.Module):

    def __init__(self, features, embed_dim, history_uv_lists, history_r_lists, aggregator, cuda="cpu", uv=True):
        super(UV_Encoder, self).__init__()
        
        self.features = features
        self.uv = uv
        self.history_uv_lists = history_uv_lists
        self.history_r_lists = history_r_lists
        self.aggregator = aggregator
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #
        
    def forward(self, nodes):
        tmp_history_uv = []
        tmp_history_r = []
        for node in nodes:
            tmp_history_uv.append(self.history_uv_lists[int(node)])
            tmp_history_r.append(self.history_r_lists[int(node)])
            
#             邻居节点的向量表示
        neigh_feats = self.aggregator.forward(nodes, tmp_history_uv, tmp_history_r)  # user-item network
#             自身节点的嵌入
        self_feats = self.features.weight[nodes]
            # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
#         线性变换
        combined = F.relu(self.linear1(combined))
        return combined

### Social_Aggregator



In [22]:
class Social_Aggregator(nn.Module):
    """
    Social Aggregator: for aggregating embeddings of social neighbors.
    """

    def __init__(self, features, u2e, embed_dim, cuda="cpu"):
        super(Social_Aggregator, self).__init__()

        self.features = features
        self.device = cuda
        self.u2e = u2e
        self.embed_dim = embed_dim
        self.att = Attention(self.embed_dim)
        
    def forward(self,nodes,to_neighs):
        
        embed_matrix = torch.empty(len(nodes), self.embed_dim, dtype=torch.float).to(self.device)
#         遍历所有节点
        for i in range(len(nodes)):
#             所有邻居节点
            tmp_adj = to_neighs[i]
            num_neighs = len(tmp_adj)
            # 邻居节点 嵌入
            e_u = self.u2e.weight[list(tmp_adj)] # fast: user embedding 
            
            #slow: item-space user latent factor (item aggregation)
            #feature_neigbhors = self.features(torch.LongTensor(list(tmp_adj)).to(self.device))
            #e_u = torch.t(feature_neigbhors)
#             中心的user的向量表示
            u_rep = self.u2e.weight[nodes[i]]
#             通过用户和邻居的向量学习alpha
            att_w = self.att(e_u, u_rep, num_neighs)
#             Aggre_{neigbhors}
            att_history = torch.mm(e_u.t(), att_w).t()
            embed_matrix[i] = att_history
        to_feats = embed_matrix

        return to_feats
        

### Social_Encoder

In [23]:
class Social_Encoder(nn.Module):

    def __init__(self, features, embed_dim, social_adj_lists, aggregator, base_model=None, cuda="cpu"):
        super(Social_Encoder, self).__init__()

        self.features = features
        self.social_adj_lists = social_adj_lists
        self.aggregator = aggregator
        if base_model != None:
            self.base_model = base_model
        self.embed_dim = embed_dim
        self.device = cuda
        self.linear1 = nn.Linear(2 * self.embed_dim, self.embed_dim)  #

    def forward(self, nodes):

        to_neighs = []
        for node in nodes:
            to_neighs.append(self.social_adj_lists[int(node)])
        
#         Aggre_{neigbhors}
        neigh_feats = self.aggregator.forward(nodes, to_neighs)  # user-user network
    
#         这里的feature是user-item传递过来的user的嵌入词典
        self_feats = self.features(torch.LongTensor(nodes.cpu().numpy())).to(self.device)
        self_feats = self_feats.t()
        
        # self-connection could be considered.
        combined = torch.cat([self_feats, neigh_feats], dim=1)
        combined = F.relu(self.linear1(combined))

        return combined

In [24]:
# user feature
    # features: item * rating
agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)
    # neighobrs
agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                        base_model=enc_u_history, cuda=device)


In [25]:
# item feature: user * rating
agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)



###  GraphRec

In [26]:
class GraphRec(nn.Module):

    def __init__(self, enc_u, enc_v_history, r2e):
        super(GraphRec, self).__init__()
        self.enc_u = enc_u
        self.enc_v_history = enc_v_history
        self.embed_dim = enc_u.embed_dim

        self.w_ur1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_ur2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr1 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_vr2 = nn.Linear(self.embed_dim, self.embed_dim)
        self.w_uv1 = nn.Linear(self.embed_dim * 2, self.embed_dim)
        self.w_uv2 = nn.Linear(self.embed_dim, 16)
        self.w_uv3 = nn.Linear(16, 1)
        self.r2e = r2e
        self.bn1 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn2 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn3 = nn.BatchNorm1d(self.embed_dim, momentum=0.5)
        self.bn4 = nn.BatchNorm1d(16, momentum=0.5)
        self.criterion = nn.MSELoss()

    def forward(self, nodes_u, nodes_v):
        embeds_u = self.enc_u(nodes_u)
        embeds_v = self.enc_v_history(nodes_v)

        x_u = F.relu(self.bn1(self.w_ur1(embeds_u)))
        x_u = F.dropout(x_u, training=self.training)
        x_u = self.w_ur2(x_u)
        x_v = F.relu(self.bn2(self.w_vr1(embeds_v)))
        x_v = F.dropout(x_v, training=self.training)
        x_v = self.w_vr2(x_v)

        x_uv = torch.cat((x_u, x_v), 1)
        x = F.relu(self.bn3(self.w_uv1(x_uv)))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.bn4(self.w_uv2(x)))
        x = F.dropout(x, training=self.training)
        scores = self.w_uv3(x)
        return scores.squeeze()

    def loss(self, nodes_u, nodes_v, labels_list):
        scores = self.forward(nodes_u, nodes_v)
        return self.criterion(scores, labels_list)


In [28]:
def train(model, device, train_loader, optimizer, epoch, best_rmse, best_mae):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        batch_nodes_u, batch_nodes_v, labels_list = data
        optimizer.zero_grad()
        loss = model.loss(batch_nodes_u.to(device), batch_nodes_v.to(device), labels_list.to(device))
        loss.backward(retain_graph=True)
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 0:
            print('[%d, %5d] loss: %.3f, The best rmse/mae: %.6f / %.6f' % (
                epoch, i, running_loss / 100, best_rmse, best_mae))
            running_loss = 0.0
    return 0


def test(model, device, test_loader):
    model.eval()
    tmp_pred = []
    target = []
    with torch.no_grad():
        for test_u, test_v, tmp_target in test_loader:
            test_u, test_v, tmp_target = test_u.to(device), test_v.to(device), tmp_target.to(device)
            val_output = model.forward(test_u, test_v)
            tmp_pred.append(list(val_output.data.cpu().numpy()))
            target.append(list(tmp_target.data.cpu().numpy()))
    tmp_pred = np.array(sum(tmp_pred, []))
    target = np.array(sum(target, []))
    expected_rmse = sqrt(mean_squared_error(tmp_pred, target))
    mae = mean_absolute_error(tmp_pred, target)
    return expected_rmse, mae


def main():
    # Training settings
    parser = argparse.ArgumentParser(description='Social Recommendation: GraphRec model')
    parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
    parser.add_argument('--embed_dim', type=int, default=64, metavar='N', help='embedding size')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR', help='learning rate')
    parser.add_argument('--test_batch_size', type=int, default=1000, metavar='N', help='input batch size for testing')
    parser.add_argument('--epochs', type=int, default=100, metavar='N', help='number of epochs to train')
    args = parser.parse_args(args=[])

    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    use_cuda = False
    if torch.cuda.is_available():
        use_cuda = True
    device = torch.device("cuda" if use_cuda else "cpu")

    embed_dim = args.embed_dim
    dir_data = './data/toy_dataset'

    path_data = dir_data + ".pickle"
    data_file = open(path_data, 'rb')
    history_u_lists, history_ur_lists, history_v_lists, history_vr_lists, train_u, train_v, train_r, test_u, test_v, test_r, social_adj_lists, ratings_list = pickle.load(
        data_file)


    trainset = torch.utils.data.TensorDataset(torch.LongTensor(train_u), torch.LongTensor(train_v),
                                              torch.FloatTensor(train_r))
    testset = torch.utils.data.TensorDataset(torch.LongTensor(test_u), torch.LongTensor(test_v),
                                             torch.FloatTensor(test_r))
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=args.test_batch_size, shuffle=True)
    num_users = history_u_lists.__len__()
    num_items = history_v_lists.__len__()
    num_ratings = ratings_list.__len__()

    u2e = nn.Embedding(num_users, embed_dim).to(device)
    v2e = nn.Embedding(num_items, embed_dim).to(device)
    r2e = nn.Embedding(num_ratings, embed_dim).to(device)

    # user feature
    # features: item * rating
    # 先通过将user-item 进行 Agger  
    agg_u_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=True)
    
    enc_u_history = UV_Encoder(u2e, embed_dim, history_u_lists, history_ur_lists, agg_u_history, cuda=device, uv=True)
    # neighobrs
    agg_u_social = Social_Aggregator(lambda nodes: enc_u_history(nodes).t(), u2e, embed_dim, cuda=device)
    enc_u = Social_Encoder(lambda nodes: enc_u_history(nodes).t(), embed_dim, social_adj_lists, agg_u_social,
                           base_model=enc_u_history, cuda=device)

    # item feature: user * rating
    agg_v_history = UV_Aggregator(v2e, r2e, u2e, embed_dim, cuda=device, uv=False)
    enc_v_history = UV_Encoder(v2e, embed_dim, history_v_lists, history_vr_lists, agg_v_history, cuda=device, uv=False)

    # model
    graphrec = GraphRec(enc_u, enc_v_history, r2e).to(device)
    optimizer = torch.optim.RMSprop(graphrec.parameters(), lr=args.lr, alpha=0.9)

    best_rmse = 9999.0
    best_mae = 9999.0
    endure_count = 0

    for epoch in range(1, args.epochs + 1):

        train(graphrec, device, train_loader, optimizer, epoch, best_rmse, best_mae)
        expected_rmse, mae = test(graphrec, device, test_loader)
        # please add the validation set to tune the hyper-parameters based on your datasets.

        # early stopping (no validation set in toy dataset)
        if best_rmse > expected_rmse:
            best_rmse = expected_rmse
            best_mae = mae
            endure_count = 0
        else:
            endure_count += 1
        print("rmse: %.4f, mae:%.4f " % (expected_rmse, mae))

        if endure_count > 5:
            break


In [ ]:
main()

[1,     0] loss: 0.105, The best rmse/mae: 9999.000000 / 9999.000000
[1,   100] loss: 8.032, The best rmse/mae: 9999.000000 / 9999.000000
rmse: 2.4514, mae:2.2948 
[2,     0] loss: 0.057, The best rmse/mae: 2.451415 / 2.294840
[2,   100] loss: 4.005, The best rmse/mae: 2.451415 / 2.294840
rmse: 1.5091, mae:1.3406 
[3,     0] loss: 0.019, The best rmse/mae: 1.509148 / 1.340588
